In [ ]:
import json
import requests
import pandas as pd
import psycopg2
import os

# Load in environmental variables

ENDPOINT = os.environ["ENDPOINT"]
END_DB_NAME = os.environ["END_DB_NAME"]
END_USERNAME = os.environ["END_USERNAME"]
END_PASSWORD = os.environ["END_PASSWORD"]
SOURCE_DB_NAME	= os.environ["SOURCE_DB_NAME"]
SOURCE_PASSWORD	= os.environ["SOURCE_PASSWORD"]
SOURCE_USERNAME	= os.environ["SOURCE_USERNAME"]
SOURCE_POINT = os.environ["SOURCE_POINT"]


# Connect to source and target databases

def lambda_handler(event, context):
    try: 
        source_conn = psycopg2.connect("host={} dbname={} user={} password={}".format(SOURCE_POINT, SOURCE_DB_NAME, SOURCE_USERNAME, SOURCE_PASSWORD))
    except psycopg2.Error as e: 
        print("Error: Could not make connection to source database")
        print(e)
        return
    
    try: 
        source_cur = source_conn.cursor()
    except psycopg2.Error as e: 
        print("Error: Could not get curser to source database")
        print(e)
        return
    
    source_conn.set_session(autocommit=True)
    
    try: 
        target_conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, END_DB_NAME, END_USERNAME, END_PASSWORD))
    except psycopg2.Error as e: 
        print("Error: Could not make connection to target database")
        print(e)
        return
    
    try: 
        target_cur = target_conn.cursor()
    except psycopg2.Error as e: 
        print("Error: Could not get curser to target database")
        print(e)
        return
    
    target_conn.set_session(autocommit=True)

# Grab latest datetime from datawarehouse - this will ensure only new data is uploaded

    try:
        target_cur.execute("SELECT MAX(datetimeto) FROM public.datetime;")
        last_datetimeto = target_cur.fetchone()[0]
    except psycopg2.Error as e:
        print("Error: Could not fetch the last datetimeto from the target database")
        print(e)
        return
    
# Access datetime table   
    try:
        source_cur.execute("""
            SELECT
                datetimefrom,
                datetimeto,
                year,
                month,
                day,
                weekday,
                hourfrom,
                date,
                timefrom,
                timeto,
                dayofyear
            FROM 
                public.datetime_info
            WHERE
            datetimeto > %s;
        """, (last_datetimeto,))
        records = source_cur.fetchall()
    except psycopg2.Error as e:
        print("Error: Could not fetch data from source database (datetime)")
        print(e)
        return
    
# Insert data into target database (datetime)
    try:
        for record in records:
            target_cur.execute("""
                INSERT INTO public.datetime (
                    datetimefrom,
                    datetimeto,
                    year,
                    month,
                    day,
                    weekday,
                    hourfrom,
                    date,
                    timefrom,
                    timeto,
                    dayofyear
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            """, record)
    except psycopg2.Error as e:
        print("Error: Could not insert data into target database (datetime)")
        print(e)
        return

# Track data transfer in logging table, makes it easier to find errors and lost data

    try:
        source_cur.execute("""
        INSERT INTO update_log (table_name, rows_transferred, latest_datetimeto)
        VALUES ('datetime', %s, %s);
    """, (len(records), last_datetimeto))
    except psycopg2.Error as e:
        print("Error: Couldn't upload tracking info (datetime)")
        print(e)
        return

# Access Location data
    try:
        source_cur.execute("""
            SELECT
                zst_id,
                sitename,
                latitude,
                longitude
            FROM 
                location_info
        """)
        records = source_cur.fetchall()
    except psycopg2.Error as e:
        print("Error: Could not fetch data from source database (location)")
        print(e)
        return
    
    
# Insert data into target database, ON CONFLICT DO NOTHING ensures that only new locations with unique
# primary keys (zst_id) are added.
    try:
        for record in records:
            target_cur.execute("""
                INSERT INTO public.location (
                    zst_id,
                    sitename,
                    latitude,
                    longitude
                )
                VALUES (%s, %s, %s, %s)
                ON CONFLICT DO NOTHING;
            """, record)
    except psycopg2.Error as e:
        print("Error: Could not insert data into target database (location)")
        print(e)
        return
    
# Again, data transfer is logged.  
    try:
        source_cur.execute("""
        INSERT INTO update_log (table_name, rows_transferred, latest_datetimeto)
        VALUES ('location', %s, %s);
    """, (len(records), last_datetimeto))
    except psycopg2.Error as e:
        print("Error: Couldn't upload tracking info (datetime)")
        print(e)
        return
    
# Access traffic info data    
    try:
        source_cur.execute("""
            SELECT
                zst_id,
                datetimefrom,
                datetimeto,
                valuesapproved,
                valuesedited,
                traffictype,
                total_sum
            FROM 
                public.traffic_agg
            WHERE
            datetimeto > %s;
        """, (last_datetimeto,))
        records = source_cur.fetchall()
    except psycopg2.Error as e:
        print("Error: Could not fetch data from source database (traffic)")
        print(e)
        return

    
# Insert data into target database (traffic)
    try:
        for record in records:
            target_cur.execute("""
                INSERT INTO public.traffic_table (
                    zst_id,
                    datetimefrom,
                    datetimeto,
                    valuesapproved,
                    valuesedited,
                    traffictype,
                    total_sum
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s);
            """, record)
    except psycopg2.Error as e:
        print("Error: Could not insert data into target database (traffic")
        print(e)
        return
    
# Log transfer   
    try:
        source_cur.execute("""
        INSERT INTO update_log (table_name, rows_transferred, latest_datetimeto)
        VALUES ('traffic', %s, %s);
    """, (len(records), last_datetimeto))
    except psycopg2.Error as e:
        print("Error: Couldn't upload tracking info (datetime)")
        print(e)
        return
    
# Close connections
    
    source_cur.close()
    source_conn.close()
    target_cur.close()
    target_conn.close()